# Data Preparation (Part 1)

In this part, the raw hdb resale flat price data is modified and combined. The dataset from this part will be used to retrieve addtional details about the location of the resale flat.

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
df2 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")
df3 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
df4 = pd.read_csv("data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")

In [3]:
df1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
df2.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [5]:
df3.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0


In [6]:
df4.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0


## Modifying Dataframe Features for Concatenation

As the four dataframes do not have the same set of features, I have chosen to modify the dataframes to have the same set of features so that it will be easier to concatenate the dataframes. The year of transaction was extracted after converting `month` to a datetime object, after which the `remaining_lease` is computed for each row. Note that each resale flat has a 99-year lease.

In [7]:
def extract_year(df):
    df["month"] = pd.to_datetime(df["month"], format="%Y-%m").dt.to_period("M")
    df.insert(loc=0, column="year", value=df["month"].dt.year)
    
    df["remaining_lease"] = 99 - (df["year"] - df["lease_commence_date"])
    return df

In [8]:
df1 = extract_year(df1)
df2 = extract_year(df2)
df3 = extract_year(df3)
df4 = extract_year(df4)

Columns of each dataframe is then reordered to have the same order so that is easier to concatenate all dataframes into a single dateframe.

In [9]:
col_order = df1.columns.to_list()
def reorder_cols(df, col_order):
    return df[col_order]

df1 = reorder_cols(df1, col_order)
df2 = reorder_cols(df2, col_order)
df3 = reorder_cols(df3, col_order)
df4 = reorder_cols(df4, col_order)

In [10]:
df_combine = pd.concat([df1, df2, df3, df4], axis=0) # concat all dfs

As I require the flat address (`street_name` + `block`) to retrieve futher information about its location, a feature, `address` is created. The data is written to a csv to be used to retrieve more information about the resale flat locaiton.

In [11]:
df_combine.insert(loc=4, column="address", value=df_combine["block"].astype(str) + " " + df_combine["street_name"].astype(str))

In [12]:
df_combine.head()

,year,month,town,flat_type,address,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017,2017-01,ANG MO KIO,2 ROOM,406 ANG MO KIO AVE 10,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,232000.0
1,2017,2017-01,ANG MO KIO,3 ROOM,108 ANG MO KIO AVE 4,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,250000.0
2,2017,2017-01,ANG MO KIO,3 ROOM,602 ANG MO KIO AVE 5,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,262000.0
3,2017,2017-01,ANG MO KIO,3 ROOM,465 ANG MO KIO AVE 10,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,265000.0
4,2017,2017-01,ANG MO KIO,3 ROOM,601 ANG MO KIO AVE 5,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,265000.0


In [14]:
df_combine.shape

(617304, 13)

In [13]:
df_combine.to_csv("data/resale-flat-prices-2000-present.csv", index=False)

# Data Preparation (Part 2)

After additional details of the resale flats have been retrieve and combined with the dataset from above, feature selection and engineering is done in this part to produce the final data to be used for the model.

MRT and LRT station location details have been retrieved from other sources, which will be combined to the HDB dataset to produce the feature, `walking distance to nearest station`.

In [ ]:
import pandas as pd

## Dropping Correlated Features

Since some features are repeated or correlated, I removed them and kept the features that would be used in the model.
- `lease_commence_date` is dropped since it is related to `remaining_lease`
- `year` is no longer needed since it was only required to calculate `remaining_lease`
- `address` is no longer needed since it was only required to retrieve the location details of the flat
- `town`, `block` and `street_name` are likely to be highly correlated as they are indicators of the flat location. I have chosen to drop `town` and `block`

In [ ]:
# df_final.drop(columns=["year", "address", "town", "block", "lease_commence_date"], inplace=True)